In [1]:
import os
os.chdir('..')
os.getcwd()

'c:\\Users\\wlsdu\\Desktop\\developer\\final_project\\topik\\topik-korea-data'

In [2]:
from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
import ast

# 유형1

- 바로 채점

In [6]:
# sample_paragraph = "초대합니다. 한 달 전에 이사를 했습니다. 그동안 집안 정리 때문에 정신이 없었는데 이제 좀 정리가 됐습니다." \
#     "그래서 저희 집에서 (㉠). (㉡)? 그 시간이 괜찮으신지 연락 주시면 감사하겠습니다."
# sample_answer1 = "파티를 열려고 합니다."
# # sample_answer2 = "5월 17일에 오후 1시에 시간되실까요"
# sample_answer2 = "음식은 양식 어떠세요"
# criteria = "답안을 작성할 때는 담화의 앞뒤 내용을 잘 파악하는 것이 중요합니다." \
#         " ㉠과 ㉡ 의 앞이나 뒤에 있는 문장들을 잘 살펴보고 내용이 자연스럽게 이어지도록 해야 합니다. "\
#         "담화의 문맥에 적합하지 않은 어휘나 문법을 사용하면 감점이 됩니다." \
#         "불필요한 내용이 추가되어 원래의 의미를 해치는 경우 감점이 됩니다."\
#         "철자법이 정확하지 않거나 글의 형식성, 격식성에 맞지 않으면 감점이 됩니다."\
#         "답안에 (      ) 앞뒤의 어구를 포함해서 쓰지 않도록 주의하십시오."\
#         "답안을 한 문장 이상으로 쓰지 않도록 주의하십시오."

# def prompt_for_writing_type_1(paragraph=None, answers=None):
#     if paragraph is None or answers is None:
#         raise ValueError("paragraph is required")
#     else:
#         sysMsg = '너는 글쓰기 평가 전문가야. 입력된 답안을 채점해줘. 빈칸별 점수 범위 0-5점. 예시: {"total_points": 8}'
#         userMsg = {"paragraph": paragraph, "blank1": answers[0], "blank2": answers[1], "답안 작성 방법": criteria}
#     return sysMsg, str(userMsg)

# verbose=False
# default_model="gpt-4o-mini"
# total_cost = 0
# models = [default_model]
# problem_generate_model = Problem_Generate_Model(
#     models, use_cache=True, temperature=0.8, verbose=verbose
# )

# sysMsg, userMsg =  prompt_for_writing_type_1(sample_paragraph, [sample_answer1, sample_answer2])

# problem_generate_model.request_models_responses(
#     [
#         SystemMessage(content=sysMsg),
#         HumanMessage(content=userMsg),
#     ]
# )
# assessment_responses = problem_generate_model.get_model_responses()
# total_points = assessment_responses[0][1]
# total_points

'{"total_points": 9}'

In [3]:
# new
from openai import OpenAI
import numpy as np
import json
import sys
api_key_file = "API_KEY/llm_api_key.json"
if os.path.exists(api_key_file):
    with open(api_key_file) as f:
        api_key = json.load(f)
else:
    api_key = {
        "OPENAI_API_KEY": "your_openai_api_key_here",
    }
    with open(api_key_file, "w") as f:
        json.dump(api_key, f)

if (
    api_key["OPENAI_API_KEY"] == "your_openai_api_key_here"
):
    print("Please update your API keys in the API_KEY/api_key.json file")
    sys.exit()
else:
    # openai.api_key = api_key["OPENAI_API_KEY"]
    client = OpenAI(
    api_key=api_key["OPENAI_API_KEY"]
    )

# Function to get the embedding
def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# Function to calculate cosine similarity
def cosine_similarity(embedding1, embedding2):
    # Cosine similarity: dot_product(A, B) / (norm(A) * norm(B))
    embedding1 = np.array(embedding1)
    embedding2 = np.array(embedding2)
    
    dot_product = np.dot(embedding1, embedding2)
    norm_a = np.linalg.norm(embedding1)
    norm_b = np.linalg.norm(embedding2)
    
    return dot_product / (norm_a * norm_b)

# Sample texts
text1 = "OpenAI is an AI research lab."
text2 = "Artificial intelligence advances in the lab."

# Get embeddings for both texts
embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)

# Calculate similarity
similarity = cosine_similarity(embedding1, embedding2)

print(f"Similarity score between text1 and text2: {similarity}")

Similarity score between text1 and text2: 0.5945091149557441


- 답변 생성 후 비교

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from difflib import SequenceMatcher
from openai import OpenAI
import numpy as np
import json
import sys

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def create_openai_client():
    api_key_file = "API_KEY/llm_api_key.json"
    if os.path.exists(api_key_file):
        with open(api_key_file) as f:
            api_key = json.load(f)
    else:
        api_key = {
            "OPENAI_API_KEY": "your_openai_api_key_here",
        }
        with open(api_key_file, "w") as f:
            json.dump(api_key, f)

    if (
        api_key["OPENAI_API_KEY"] == "your_openai_api_key_here"
    ):
        print("Please update your API keys in the API_KEY/api_key.json file")
        sys.exit()
    else:
        # openai.api_key = api_key["OPENAI_API_KEY"]
        client = OpenAI(
        api_key=api_key["OPENAI_API_KEY"]
        )
    return client

# Function to get the embedding
def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# Function to calculate cosine similarity
def cosine_similarity(embedding1, embedding2):
    # Cosine similarity: dot_product(A, B) / (norm(A) * norm(B))
    embedding1 = np.array(embedding1)
    embedding2 = np.array(embedding2)
    
    dot_product = np.dot(embedding1, embedding2)
    norm_a = np.linalg.norm(embedding1)
    norm_b = np.linalg.norm(embedding2)
    
    return dot_product / (norm_a * norm_b)

def cosine_similarity_using_openai_embeddings(text1, text2):
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity

def sentence_to_tokens(sentence):
    return word_tokenize(sentence)

def sequenceMatcher_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def filter_nouns(tokens):
    tagged_tokens = pos_tag(tokens)
    nouns = [word for word, pos in tagged_tokens if pos.startswith('NN')]
    return nouns

def similarity_between_sentences(sentence1, sentence2):
    tokens1 = sentence_to_tokens(sentence1)
    tokens2 = sentence_to_tokens(sentence2)
    nouns1 = filter_nouns(tokens1)
    nouns2 = filter_nouns(tokens2)
    similarity = 0.0
    for noun1 in nouns1:
        for noun2 in nouns2:
            similarity += sequenceMatcher_similarity(noun1, noun2)
    return similarity

def bonus_point(similarity):
    print(f"similarity: {similarity}")
    bonus_point = 0
    if similarity > 0.4 and similarity < 1.0:
        bonus_point = 2
    elif similarity >= 1.0:
        bonus_point = 4
    return bonus_point

# Example usage
sentence = "This is an example sentence."
tokens = sentence_to_tokens(sentence)
print(tokens)



['This', 'is', 'an', 'example', 'sentence', '.']


In [12]:
sample_paragraph = "초대합니다. 한 달 전에 이사를 했습니다. 그동안 집안 정리 때문에 정신이 없었는데 이제 좀 정리가 됐습니다." \
    "그래서 저희 집에서 (㉠). (㉡)? 그 시간이 괜찮으신지 연락 주시면 감사하겠습니다."
sample_answer1 = "파티를 열려고 합니다."
sample_answer2 = "5월 17일에 오후 1시에 시간되실까요"
# sample_answer2 = "5월 17일에 오후 1시에 시강되실까요"
sample_answer2 = "음식은 양식 어떠세요"
# sample_answer2 = "옴식은 양식 어떠세요"
criteria = "상: 의미가 완전히 일치함, 중: 의미가 일부 일치함, 하: 의미가 일치하지 않음"
level_to_points = {"상": 5, "중": 3, "하": 1}

verbose=False
default_model="gpt-4o-mini"
total_cost = 0
models = [default_model]
answer_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose
)

### Generate answers
answer_generate_model.request_models_responses(
    [
        # SystemMessage(content="너는 한국어 글쓰기 전문가야. 빈칸에 들어가야 할 말을 적합한 어휘를 사용하여 구체적으로 채워넣어줘. 예시: {'blank1': '', 'blank2': ''}"),
        SystemMessage(content="너는 한국어 전문가야. 빈칸에 들어가야 할 말을 문맥에 맞게, 구체적으로 생성해줘. 예시: {'answer1': '', 'answer2': ''}"),
        HumanMessage(content=f"paragraph: {sample_paragraph}"),
    ]
)
answer_responses = answer_generate_model.get_model_responses()
answers = answer_responses[0][1]
answers = ast.literal_eval(answers)
display(answers)

### Assess answers based on the criteria, which is 상/중/하
answer_assessment_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.8, verbose=verbose
)

userMsg = {"answer1:": answers["answer1"], "answer1_guess": sample_answer1, 
           "answer2:": answers["answer2"], "answer2_guess": sample_answer2}
userMsg = str(userMsg)
response_example = "예시: {'point1': '상', 'point2': '중'}"
print(f"userMsg: {userMsg}")
answer_assessment_model.request_models_responses(
    [
        # SystemMessage(content="정답과 비교하여 입력된 답안을 채점해줘. 의미만 유사하면 정답이야. 정답과 입력에 포함되어있는 숫자는 달라도 돼. 후하게 점수를 줘. 입력별 점수 범위 0-5점. 예시: {'total_points': 8, '채점 근거': ''}"),
        # SystemMessage(content="정답과 비교하여 입력된 답안을 채점해줘. generous하게 점수를 줘. 총점 범위 0-10점. 예시: {'total_points': 8, '채점 근거': ''}"),
        SystemMessage(content=f"정답과 비교하여 입력된 답안을 채점해줘. 채점 기준: {criteria}. 예시: {response_example}"),
        HumanMessage(content=userMsg),
    ]
)
scoring_responses = answer_assessment_model.get_model_responses()
scoring_result = scoring_responses[0][1]
scoring_result = ast.literal_eval(scoring_result)
display(scoring_result)
point_pair = [level_to_points[scoring_result["answer1"]], level_to_points[scoring_result["answer2"]]]
# print(f"scoring_result: {scoring_result}")
# total_points = scoring_result["total_points"]
# # total_points += 2
# if total_points > 10:
#     total_points = 10
# print(f"total_points: {total_points}")

### post-processing
## Translate the answers to English
translate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.0, verbose=verbose
)
translate_model.request_models_responses(
    [
        SystemMessage(content="너는 번역가야. 한국어를 영어로 번역해줘. 예시: {'answer1': '', 'answer1_guess': '', 'answer2': '', 'answer2_guess': ''}"),
        HumanMessage(content=f"input: {userMsg}"),
    ]
)
english_responses = translate_model.get_model_responses()
english_answers = english_responses[0][1]
english_answers = ast.literal_eval(english_answers)
display(english_answers)

similarity1 = cosine_similarity_using_openai_embeddings(english_answers["answer1"], english_answers["answer1_guess"])
similarity2 = cosine_similarity_using_openai_embeddings(english_answers["answer2"], english_answers["answer2_guess"])

point_pair[0] += bonus_point(similarity1)
point_pair[1] += bonus_point(similarity2)
if point_pair[0] > 5:
    point_pair[0] = 5
if point_pair[1] > 5:
    point_pair[1] = 5
print(f"point_pair: {point_pair}")

### Grammar checking model
grammar_check_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.3, verbose=verbose
)
grammar_check_model.request_models_responses(
    [
        SystemMessage(content="당신은 오타(typo)를 찾아내는 도우미입니다. 주어진 문장에서 문맥, 문장부호, 조사 유무는 고려하지 말고, 오직 단어의 철자가 정확한지 판단해주세요. 예시: {'typo_in_sentence1': 'none', 'typo_in_sentence2': '공뷰'}"),
        HumanMessage(content=f"sentence1: {sample_answer1}, sentence2: {sample_answer2}"),
    ]
)
grammar_check_responses = grammar_check_model.get_model_responses()
grammar_answers = grammar_check_responses[0][1]
grammar_answers = ast.literal_eval(grammar_answers)
grammar_answers
print(f"Grammar answers: {grammar_answers}")
if grammar_answers["typo_in_sentence1"] != "none":
    point_pair[0] -= 1
if grammar_answers["typo_in_sentence2"] != "none":
    point_pair[1] -= 1
total_points = sum(point_pair)
print(f"point_pair: {point_pair}")
print(f"total_points: {total_points}")

{'answer1': '작은 집들이를 하고 싶습니다', 'answer2': '그날 오후 6시에 괜찮으신가요'}

userMsg: {'answer1:': '작은 집들이를 하고 싶습니다', 'answer1_guess': '파티를 열려고 합니다.', 'answer2:': '그날 오후 6시에 괜찮으신가요', 'answer2_guess': '음식은 양식 어떠세요'}


{'answer1': '하', 'answer1_guess': '하', 'answer2': '하', 'answer2_guess': '하'}

{'answer1': 'I want to have a small gathering.',
 'answer1_guess': 'I am planning to hold a party.',
 'answer2': 'Is 6 PM that day okay for you?',
 'answer2_guess': 'How about Western food?'}

similarity: 0.5840664007390043
similarity: 0.20967549439498678
point_pair: [3, 1]
Grammar answers: {'typo_in_sentence1': 'none', 'typo_in_sentence2': 'none'}
point_pair: [3, 1]
total_points: 4


# 유형2

In [8]:
assessment_criteria_2_1_1 = "주어진 과제를 충실히 수행하였는가?"
assessment_criteria_2_1_2 = "주제와 관련된 내용으로 구성하였는가?"
assessment_criteria_2_1_3 = "주어진 내용을 풍부하고 다양하게 표현하였는가?"
assessment_criteria_2_2_1 = "글의 구성이 명확하고 논리적인가?"
assessment_criteria_2_2_2 = "글의 내용에 따라 단락 구성이 잘 이루어졌는가?"
assessment_criteria_2_2_3 = "논리 전개에 도움이 되는 담화 표지를 적절하게 사용하여 조직적으로 연결하였는가?"
assessment_criteria_2_3_1 = "문법과 어휘를 다양하고 풍부하게 사용하며 적절한 문법과 어휘를 선택하여 사용하였는가?"
assessment_criteria_2_3_2 = "문법, 어휘, 맞춤법 등의 사용이 정확한가?"
assessment_criteria_2_3_3 = "글의 목적과 기능에 따라 격식에 맞게 글을 썼는가?"

In [14]:
level_to_points_criteria_for_2_1 = {"상": 7/3, "중": 5/3, "하": 2/3}
level_to_points_criteria_for_2_2 = {"상": 7/3, "중": 5/3, "하": 2/3}
level_to_points_criteria_for_2_3 = {"상": 16/3, "중": 12/3, "하": 6/3}

def assess_by_criteria(llm_model, topic, user_answer, criteria, level_to_points):
    llm_model.request_models_responses(
        [
            SystemMessage(content="너는 주어진 기준에 따라 글을 채점해줘. 채점은 상/중/하 중 하나로. 예시: {'level': '상'}"),
            HumanMessage(content=f"user_answer: {user_answer}, topic: {topic}, criteria: {criteria}"),
        ]
    )
    responses = llm_model.get_model_responses()
    result = responses[0][1]
    result = ast.literal_eval(result)
    point = level_to_points[result["level"]]
    print(f"point: {point}")
    return point

In [12]:
sample_article = "인터넷은 우리 일상에 깊숙이 자리 잡은 필수적인 도구가 되었습니다. 정보의 바다라 불리는 인터넷은 전 세계의 지식을 손쉽게 접할 수 있게 해주며, 시공간의 제약 없이 소통을 가능케 합니다. 교육, 비즈니스, 엔터테인먼트 등 다양한 분야에서 혁신을 이끌어내고 있죠. 그러나 인터넷에는 부작용도 있습니다. 개인정보 유출, 사이버 범죄, 허위정보 확산 등의 문제가 끊임없이 제기되고 있습니다. 또한 과도한 사용으로 인한 중독과 현실과의 괴리 현상도 우려됩니다. 따라서 인터넷의 장점을 최대한 활용하면서도 그 부작용을 최소화하기 위한 노력이 필요합니다. 디지털 리터러시 교육과 적절한 규제, 그리고 개인의 자제력이 요구되는 시대입니다."
sample_article = "인터넷은 좋은 점도 있고 나쁜 점도 있습니다. 좋은 점은 정보를 쉽게 찾을 수 있고 멀리 있는 사람들과 대화할 수 있다는 거에요. 그리고 온라인 쇼핑도 할 수 있어서 편리합니다. 하지만 나쁜 점도 있어요. 가짜 뉴스가 많이 퍼지고 있고, 어린이들이 나쁜 것을 보게 될 수도 있습니다. 그리고 너무 많이 사용하면 중독될 수 있어요. 그래서 인터넷을 사용할 때는 조심해야 해요. 부모님들은 아이들이 인터넷을 사용할 때 옆에 있어주는 게 좋습니다. 그리고 모든 사람들이 인터넷에서 본 정보가 진짜인지 확인해봐야 해요."
sample_article = "인터넷은 좋은 점도 있고 나쁜 점도 있습니다. 좋은 점은 정보를 쉽게 찾을 수 있고 멀리 있는 사람들과 대화할 수 있다는 거에요. 그리고 온라인 쇼핑도 할 수 있어서 편리합니다. "
sample_topic = '인터넷의 장단점'
assessment_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose
)

total_points = 0
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_1_1, level_to_points_criteria_for_2_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_1_2, level_to_points_criteria_for_2_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_1_3, level_to_points_criteria_for_2_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_2_1, level_to_points_criteria_for_2_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_2_2, level_to_points_criteria_for_2_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_2_3, level_to_points_criteria_for_2_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_3_1, level_to_points_criteria_for_2_3)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_3_2, level_to_points_criteria_for_2_3)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_2_3_3, level_to_points_criteria_for_2_3)

penalty_weight = 1
if len(sample_article) < 200:
    penalty_weight = 2/3

total_points = total_points * penalty_weight
total_points = round(total_points)

if total_points > 30:
    total_points = 30
print(f"Article length: {len(sample_article)}")
print(f"total_points: {total_points}")

point: 1.6666666666666667
point: 2.3333333333333335
point: 1.6666666666666667
point: 1.6666666666666667
point: 1.6666666666666667
point: 1.6666666666666667
point: 4.0
point: 5.333333333333333
point: 5.333333333333333
Article length: 100
total_points: 17


# 유형3

In [13]:
assessment_criteria_3_1_1 = "주어진 과제를 충실히 수행하였는가?"
assessment_criteria_3_1_2 = "주제와 관련된 내용으로 구성하였는가?"
assessment_criteria_3_1_3 = "내용을 풍부하고 다양하게 표현하였는가?"
assessment_criteria_3_2_1 = "글의 구성이 명확하고 논리적인가?"
assessment_criteria_3_2_2 = "중심 생각이 잘 구성되어 있는가?"
assessment_criteria_3_2_3 = "논리 전개에 도움이 되는 담화 표지를 적절하게 사용하여 조직적으로 연결하였는가?"
assessment_criteria_3_3_1 = "문법과 어휘를 다양하고 풍부하게 사용하며 적절한 문법과 어휘를 선택하여 사용하였는가?"
assessment_criteria_3_3_2 = "문법, 어휘, 맞춤법 등의 사용이 정확한가?"
assessment_criteria_3_3_3 = "글의 목적과 기능에 따라 격식에 맞게 글을 썼는가?"

level_to_points_criteria_for_3_1 = {"상": 12/3, "중": 8/3, "하": 4/3}
level_to_points_criteria_for_3_2 = {"상": 12/3, "중": 8/3, "하": 4/3}
level_to_points_criteria_for_3_3 = {"상": 26/3, "중": 18/3, "하": 10/3}

In [17]:
sample_article = "환경오염을 줄이는 것은 우리 세대의 가장 중요한 과제 중 하나입니다. 지구 온난화, 대기오염, 해양 플라스틱 문제 등 다양한 환경 문제가 심각해지고 있는 현 시점에서, 우리 모두가 적극적으로 행동에 나서야 할 때입니다. 먼저, 개인적 차원에서 할 수 있는 일부터 시작해야 합니다. 일회용품 사용을 줄이고 재사용 가능한 제품을 선택하는 것이 중요합니다. 예를 들어, 플라스틱 빨대 대신 금속 빨대를, 비닐봉지 대신 에코백을 사용하는 등의 작은 실천이 모여 큰 변화를 만들 수 있습니다. 또한, 에너지 절약을 위해 불필요한 전등을 끄고, 대중교통이나 자전거 이용을 늘리는 것도 효과적인 방법입니다. 기업들의 역할도 중요합니다. 생산 과정에서 발생하는 오염물질을 줄이고, 친환경 기술 개발에 투자해야 합니다. 재생 에너지 사용을 확대하고, 제품 포장을 최소화하는 등의 노력이 필요합니다. 또한, 기업의 사회적 책임을 다하기 위해 환경보호 활동에 적극적으로 참여하고, 소비자들에게 환경 친화적인 선택을 할 수 있도록 정보를 제공해야 합니다. 정부 차원에서는 강력한 환경 정책과 법규를 마련하고 시행해야 합니다. 오염 물질 배출 기준을 강화하고, 위반 시 엄중한 처벌을 해야 합니다. 또한, 친환경 기술 개발을 위한 연구비 지원, 재활용 인프라 구축, 친환경 제품 사용 촉진을 위한 세제 혜택 등 다양한 정책을 통해 환경보호를 장려해야 합니다. 교육의 중요성도 간과할 수 없습니다. 어릴 때부터 환경 보호의 중요성을 인식하고 실천할 수 있도록 학교와 가정에서 체계적인 환경 교육이 이루어져야 합니다. 이를 통해 미래 세대가 환경에 대한 책임감을 가지고 지속 가능한 발전을 이끌어갈 수 있을 것입니다. 결론적으로, 환경오염을 줄이기 위해서는 개인, 기업, 정부, 교육 기관 등 사회 전반의 협력과 노력이 필요합니다. 우리 모두가 환경 보호의 주체라는 인식을 가지고, 각자의 위치에서 할 수 있는 일들을 실천해 나간다면, 보다 깨끗하고 건강한 지구를 후대에 물려줄 수 있을 것입니다. 지금 당장 시작하는 작은 변화가 미래를 바꿀 수 있다는 믿음으로, 우리 모두 환경 보호에 동참합시다."
sample_article = "환경오염은 우리 일상에 깊숙이 자리 잡은 필수적인 도구가 되었습니다. 정보의 바다라 불리는 환경오염은 전 세계의 지식을 손쉽게 접할 수 있게 해주며, 시공간의 제약 없이 소통을 가능케 합니다. 교육, 비즈니스, 엔터테인먼트 등 다양한 분야에서 혁신을 이끌어내고 있죠. 그러나 환경오염에는 부작용도 있습니다. 개인정보 유출, 사이버 범죄, 허위정보 확산 등의 문제가 끊임없이 제기되고 있습니다. 또한 과도한 사용으로 인한 중독과 현실과의 괴리 현상도 우려됩니다. 따라서 환경오염의 장점을 최대한 활용하면서도 그 부작용을 최소화하기 위한 노력이 필요합니다. 디지털 리터러시 교육과 적절한 규제, 그리고 개인의 자제력이 요구되는 시대입니다."
sample_topic = '최근 세계적으로 환경오염을 줄이기 위해 많은 노력을 기울이고 있습니다. 환경오염을 줄일 수 있는 효과적인 방법에 대해 아래의 내용을 중심으로 주장하는 글을 쓰십시오.'
assessment_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.5, verbose=verbose
)

total_points = 0
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_1_1, level_to_points_criteria_for_3_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_1_2, level_to_points_criteria_for_3_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_1_3, level_to_points_criteria_for_3_1)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_2_1, level_to_points_criteria_for_3_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_2_2, level_to_points_criteria_for_3_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_2_3, level_to_points_criteria_for_3_2)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_3_1, level_to_points_criteria_for_3_3)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_3_2, level_to_points_criteria_for_3_3)
total_points += assess_by_criteria(assessment_model, sample_topic, sample_article, assessment_criteria_3_3_3, level_to_points_criteria_for_3_3)

total_points = round(total_points)

penalty_weight = 1
if len(sample_article) < 400:
    penalty_weight = 0.0
elif len(sample_article) >= 400 and len(sample_article) < 433:
    penalty_weight = 0.5
elif len(sample_article) >= 433 and len(sample_article) < 466:
    penalty_weight = 0.666
elif len(sample_article) >= 466 and len(sample_article) < 500:
    penalty_weight = 0.833
elif len(sample_article) >= 500:
    penalty_weight = 1.0

total_points = total_points * penalty_weight

if total_points > 50:
    total_points = 50
print(f"Article length: {len(sample_article)}")
print(f"total_points: {total_points}")

point: 1.3333333333333333
point: 1.3333333333333333
point: 1.3333333333333333
point: 1.3333333333333333
point: 1.3333333333333333
point: 1.3333333333333333
point: 3.3333333333333335
point: 3.3333333333333335
point: 3.3333333333333335
Article length: 355
total_points: 0.0
